# SpatialAgent (Local LLM)

Run SpatialAgent with locally-served LLMs — no API keys required.

**Architecture**: Plan → Act → Conclude workflow with LangGraph

## Step 1: Start Local LLM Servers

Before running the agent, start the local LLM servers in a terminal:

```bash
# One-time setup (only needed once)
./local_llm/vllm/setup.sh

# Start servers (Qwen3-VL-32B on 2x A100s)
./local_llm/vllm/start.sh

# Check status
./local_llm/vllm/start.sh status

# Stop when done
./local_llm/vllm/start.sh stop
```

## Step 2: Set Environment Variables

Run this cell to point SpatialAgent to the local servers.

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

# Point to local LLM servers
os.environ["CUSTOM_MODEL_BASE_URL"] = "http://localhost:8088/v1"
os.environ["CUSTOM_EMBED_BASE_URL"] = "http://localhost:8088/v1"
os.environ["CUSTOM_EMBED_MODEL"] = "qwen3-embedding"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from spatialagent.agent import SpatialAgent, make_llm

# Initialize LLM (uses local Qwen3-VL-32B via LiteLLM proxy)
llm = make_llm("qwen3-vl-32b")

# Initialize agent
agent = SpatialAgent(llm=llm, save_path="./experiments/local/")

Auto-loading tools from tool modules...
  Loaded: download_czi_reference (database)
  Loaded: extract_czi_markers (database)
  Loaded: query_celltype_genesets (database)
  Loaded: query_disease_genes (database)
  Loaded: query_tissue_expression (database)
  Loaded: search_cellmarker2 (database)
  Loaded: search_czi_datasets (database)
  Loaded: search_panglao (database)
  Loaded: validate_genes_expression (database)
  Loaded: extract_pdf_content (literature)
  Loaded: extract_url_content (literature)
  Loaded: fetch_supplementary_from_doi (literature)
  Loaded: query_arxiv (literature)
  Loaded: query_pubmed (literature)
  Loaded: search_semantic_scholar (literature)
  Loaded: web_search (literature)
  Loaded: aggregate_gene_voting (analytics)
  Loaded: cell2location_mapping (analytics)
  Loaded: cellphonedb_analysis (analytics)
  Loaded: cellphonedb_degs_analysis (analytics)
  Loaded: cellphonedb_filter (analytics)
  Loaded: cellphonedb_plot (analytics)
  Loaded: cellphonedb_prepare (

## Step 3: Run the Agent

### Example 1: Gene Panel Design

In [2]:
result = agent.run(
    """Design a 50-gene panel for mouse prostate cancer models that captures tumor state, 
    immune process, and tissue context."""
)

<user query>
Design a 50-gene panel for mouse prostate cancer models that captures tumor state, 
    immune process, and tissue context.
</user query>

<skill> retrieved panel_design </skill>

<skill-tools> query_pubmed; search_panglao; extract_czi_markers; query_tissue_expression; search_czi_datasets; query_celltype_genesets; validate_genes_expression </skill-tools>

<tool> selected query_pubmed; execute_bash; search_panglao; extract_czi_markers; query_tissue_expression; search_czi_datasets; query_celltype_genesets; inspect_tool_code; validate_genes_expression; execute_python; web_search; search_cellmarker2; scanpy_score_genes; report_subagent; aggregate_gene_voting; query_disease_genes; query_arxiv; summarize_tissue_regions; search_semantic_scholar; verification_subagent; squidpy_ligrec; harmony_transfer_labels; fetch_supplementary_from_doi; squidpy_nhood_enrichment; download_czi_reference; summarize_celltypes </tool>

I need to design a 50-gene panel for mouse prostate cancer models

INFO:gget.utils:Performing Enrichr analysis using database PanglaoDB_Augmented_2021.


<observation>
Output:
Found 5 papers on PubMed:

Paper 1:
  Title: ERG-driven prostate cancer initiation is cell-context dependent and requires KMT2A and DOT1L.
  Journal: Nature genetics
  PubMed ID: 40858905
  Abstract: Despite the high prevalence of ERG transcription factor translocations in prostate cancer, the mechanism of tumorigenicity remains poorly understood. Using lineage tracing, we find the tumor-initiating activity of ERG resides in a subpopulation of murine basal cells that coexpress luminal genes (BasalLum) and not in the larger population of ERG+ luminal cells. Upon ERG activation, BasalLum cells give rise to highly proliferative intermediate (IM) cells with stem-like features that coexpress basal, luminal, hillock and club marker genes, before transitioning to Krt8+ luminal cells. Transcriptomic analysis of ERG+ human prostate cancers confirms the presence of rare ERG+ BasalLum cells, as well as IM cells whose presence is associated with a worse prognosis. Single-cell

INFO:gget.utils:Fetching the tissue expression atlas of KRT5 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of KRT14 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of KRT8 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of KRT18 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of PAX8 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of NKX3-1 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of SOX9 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of ACTA2 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of COL1A1 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of PECAM1 from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of CD3E from human ARCHS4 data.
INFO:gget.utils:Fetching the tissue expression atlas of 

<observation>
Output:
[extract_czi_markers] Processing a13bda79-9134-46c9-9ed1-a2858be9aafe...
Successfully processed 1 CZI dataset(s) with 12 cell types. Saved to ./experiments/czi_reference_celltype_1.csv
Successfully processed 1 CZI dataset(s) with 12 cell types. Saved to ./experiments/czi_reference_celltype_1.csv
PanglaoDB Results (Mm, prostate):

epithelial cell (matched: Epithelial cells):
  Marker genes (8): ['KAP', 'PRSS32', 'MUC3', 'OIT1', 'ZFP322A', 'ZFP704', 'SAA3', 'SPRR2A1']

luminal cell (matched: Luminal epithelial cells):
  Marker genes (43): ['FGFR2', 'FGG', 'KRT18', 'SLPI', 'PROM1', 'KRT19', 'SYTL2', 'CD74', 'AGR2', 'LTF', 'SAA2', 'KRT23', 'WFDC2', 'LCN2', 'BTG1', 'CLDN4', 'ANXA1', 'HMGA1', 'STC2', 'AREG', 'TNFSF10', 'PIP', 'ATP7B', 'HIF1A', 'FGFR4', 'DDR1', 'CEBPD', 'PGR', 'KRT8', 'UXT', 'PTH1R', 'CD9', 'AR', 'AQP3', 'ATP2C2', 'WNT5A', 'SLC12A2', 'ESR1', 'AQP5', 'RUNX1', 'CDH1', 'MUC1', 'ANPEP']

basal cell (matched: Basal cells):
  Marker genes (50): ['KRT5', 'KRT17

I have successfully designed a 50-gene panel for mouse prostate cancer models that captures tumor state, immune    
process, and tissue context. The panel is balanced with 20 genes for tumor cells, 15 genes for immune cells, and 15
genes for tissue context.                                                                                          

The genes were selected from multiple sources (CZI reference dataset, PanglaoDB, CellMarker2, and literature) and  
include well-established markers for prostate cancer and tissue context. Each gene has a detailed reason explaining
its biological function and source.                                                                                

The final panel is saved to "./experiments/final_gene_panel.csv" and includes:                                     

 • Tumor Cells: ERG, NKX3-1, PAX8, AR, KRT5, KRT14, KRT8, KRT18, SOX9, Bmi1, Trp63, KRT19, CLDN4, CDH1, MUC1, AGR2,
   LTF, SLPI, FGFR2, PROM1                                                                                         
 • Immune Cells: CD3E, CD19, CD68, PTPRC, CCL6, GIMAP3, H2-Q7, TRBC1, MS4A4C, TCRG-C1, H2-T3, TRDV4, CCL9, WFDC17, 
   H2-DMA                                                                                                          
 • Tissue Context: ACTA2, COL1A1, PECAM1, VIM, MMP2, MMP9, TLR3, MME, ITGA4, ITGAV, ICAM1, VCAM1, KIT, PDGFRA,     
   PDGFRB                                                                                                          

This panel provides comprehensive coverage of the key cell types and processes in mouse prostate cancer models and 
should be suitable for spatial transcriptomics analysis.

</conclude>


Cost Summary (qwen3-vl-32b)
Total calls:    9
Input tokens:   139,816
Output tokens:  11,495
Total tokens:   151,311
Total cost:     $0.3026



### Example 2: Cell Type Annotation

Uses thread-based memory to continue conversations.

In [ ]:
result = agent.run(
    """
    I have a MERFISH mouse liver dataset at './data/example_merfish.h5ad'.
    
    Please:
    1. Load and preprocess the data (normalize, find variable genes)
    3. Annotate cell types
    3. Run UTAG spatial clustering to identify tissue regions
    4. Generate a summary report of the tissue composition
    """,
    config={"thread_id": "annotation_demo"}
)